In [ ]:
# ライブラリの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import scipy.linalg
import itertools
import time
import torch
import torch.nn as nn
import torch.optim as optimizers
from scipy.stats import norm
from numpy.random import *
from scipy import optimize

#np.random.seed(9837)
#torch.manual_seed(9837)
pd.set_option("display.max_rows", 250)
pd.set_option("display.max_columns", 100)

In [ ]:
# 多項分布の乱数を生成する関数
def rmnom(pr, n, k, pattern):
    if pattern==1:
        z_id = np.array(np.argmax(np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis], axis=1), dtype="int")
        Z = np.diag(np.repeat(1, k))[z_id, ]
        return z_id, Z
    z_id = np.array(np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1), dtype="int")
    return z_id

# ディリクリ分布の乱数を生成する関数
def Dirichlet(alpha, n):
    x = torch.Tensor(np.random.dirichlet(alpha, n))
    return x

# データの生成

## 入力データの定義

In [ ]:
# データの設定
# パラメータ数を定義
syntax1 = 7
syntax2 = np.repeat(7, syntax1)
k = 15
d = 5000
v1 = 800
v2 = 200
v = v1 + v2

# 文書データの統計量を生成
min_word = 2
max_word = 50
pt = np.random.poisson(np.random.gamma(10.0, 1.0, d), d)
pt[pt < 5] = 5
L = np.sum(pt)
ph = np.random.poisson(np.random.gamma(17.5, 0.5, L), L)
ph[ph < 2] = 2
M = np.sum(ph)
w = np.random.poisson(np.random.gamma(75.0, 0.05, M), M)
w[w < min_word] = min_word
N = np.sum(w)

# 行列演算ベクトルを定義
k_vec = np.repeat(1.0, k)
index_k = np.arange(k)
index_v1 = np.arange(v1)
index_v2 = np.arange(v2) + v1

In [ ]:
# idとインデックスを定義
# 文書idを定義
d_id1 = np.repeat(np.repeat(np.arange(d), pt), ph)
sentence_id1 = np.repeat(np.arange(L), ph)
pt_id1 = np.hstack(([np.arange(ph[i]) for i in range(L)]))
d_id2 = np.repeat(np.repeat(np.repeat(np.arange(d), pt), ph), w)
sentence_id2 = np.repeat(np.repeat(np.arange(L), ph), w)
phrase_id2 = np.repeat(np.arange(M), w)
pt_id2 = np.hstack(([np.arange(w[i]) for i in range(M)]))

# 文書のインデックスを定義
d_list1 = []
d_list2 = []
sentence_list1 = []
sentence_list2 = []
for i in range(d):
    d_list1.append(np.where(d_id1==i)[0].astype("int"))
    d_list2.append(np.where(d_id2==i)[0].astype("int"))
for i in range(L):
    sentence_list1.append(np.where(sentence_id1==i)[0].astype("int"))
    sentence_list2.append(np.where(sentence_id2==i)[0].astype("int"))
    
# 語順のインデックスを定義
max_pt1 = np.max(pt_id1) + 1
max_pt2 = np.max(pt_id2) + 1
pt_list11 = [j for j in range(max_pt1)]
pt_list10 = [j for j in range(max_pt1)]
pt_list21 = [j for j in range(max_pt2)]
pt_list20 = [j for j in range(max_pt2)]
pt_n1 = np.repeat(0, max_pt1)
pt_n2 = np.repeat(0, max_pt2)
for j in range(max_pt1):
    pt_list11[j] = np.array(np.where(pt_id1==j)[0], dtype="int")
    if j > 0:
        pt_list10[j] = pt_list11[j] - 1
    pt_n1[j] = pt_list11[j].shape[0]
for j in range(max_pt2):
    pt_list21[j] = np.array(np.where(pt_id2==j)[0], dtype="int")
    if j > 0:
        pt_list20[j] = pt_list21[j] - 1
    pt_n2[j] = pt_list21[j].shape[0]

In [ ]:
# 機能語を生成
# フレーズのインデックスを定義
phrase_list = []
for i in range(M):
    if i==0:
        phrase_list.append(np.arange(w[i]))
    else:
        phrase_list.append(np.max(phrase_list[i-1]) + np.arange(w[i]) + 1)
        
# 機能語の位置を定義
weights = 1.0
function_flag = np.repeat(0, N)
for i in range(M):
    x = np.arange(w[i])[::-1][:3]
    logit = np.exp(weights * x)
    prob = logit / np.sum(logit)
    index = x[np.random.multinomial(1, prob, 1).reshape(-1)==1]
    function_flag[phrase_list[i][index]] = 1
function_index = np.where(function_flag==1)[0].astype("int")

## パラメータと応答変数を生成

In [ ]:
# 事前分布の定義
# HMMの事前分布を定義
alpha1 = np.repeat(0.5, syntax1)
alpha21 = np.array([0.3, 3.0])
alpha22 = np.array([1.75, 1/1.75])
beta1 = [np.repeat(0.5, syntax2[j]) for j in range(syntax1)]
beta2 = [np.repeat(0.2, syntax2[j]) for j in range(syntax1)]

# 単語分布の事前分布
gamma1 = [np.repeat(0.02, v1) for j in range(syntax1)]
gamma2 = [np.repeat(0.02, v2) for j in range(syntax1)]

In [ ]:
# パラメータを生成
# 推移確率のパラメータを生成
theta1 = np.random.gamma(alpha21[0], alpha21[1], syntax1*syntax1).reshape(syntax1, syntax1)
theta2 = np.random.gamma(alpha22[0], alpha22[1], v2*syntax1).reshape(v2, syntax1)
pi1 = np.random.dirichlet(alpha1, 1)
pi2 = np.zeros((syntax1, syntax1, v2))
for j in range(v2):
    logit = theta1 * theta2[j, ]
    pi2[:, :, j] = logit / np.sum(logit, axis=1)[:, np.newaxis]
psi1 = [np.random.dirichlet(beta1[j], 1).reshape(-1) for j in range(syntax1)]
psi2 = [np.random.dirichlet(beta2[j], syntax2[j]) for j in range(syntax1)]
thetat1 = theta1.copy(); thetat2 = theta2.copy()
pit1 = pi1.copy(); pit2 = pi2.copy(); psit1 = psi1.copy(); psit2 = psi2.copy()

# 単語分布のパラメータを生成
phi1 = []; phi2 = []; phi = []
for j in range(syntax1):
    phi1.append(np.random.dirichlet(gamma1[j], syntax2[j]))
    phi2.append(np.random.dirichlet(gamma2[j], syntax2[j]))
    phi.append(np.hstack((phi1[j], phi2[j])))
phit = phi.copy(); phit1 = phi1.copy(); phit2 = phi2.copy()

In [ ]:
# 応答変数を生成
# 生成したデータの格納用配列
Z1 = np.zeros((M, syntax1), dtype="int")
Z2 = np.zeros((N, np.max(syntax2)), dtype="int")
z1 = np.repeat(0, M)
z2 = np.repeat(0, N)
word_id = np.repeat(0, N)
function_word = np.repeat(0, M)

# フレーズごとに潜在変数と単語を生成
for i in range(M):
    if i%10000==0:
        print(i)
    
    # 上位階層の潜在変数を生成
    if pt_id1[i]==0:
        Z1[i, ] = np.random.multinomial(1, pi1.reshape(-1), 1).reshape(-1)
        z1[i] = np.argmax(Z1[i, ])
    else:
        Z1[i, ] = np.random.multinomial(1, pi2[z1[i-1], :, function_word[i-1]], 1).reshape(-1)
        z1[i] = np.argmax(Z1[i, ])

    # 下位階層の潜在変数と単語を生成
    # 上位階層に対応するパラメータを抽出
    index = phrase_list[i]
    psi01 = psi1[z1[i]]; psi02 = psi2[z1[i]]
    phi01 = phi1[z1[i]]; phi02 = phi2[z1[i]]

    # 単語単位で潜在変数と対応する単語を生成
    for j in range(w[i]):

        # 1単語目の応答変数を生成
        if j==0:
            # 下位階層の潜在変数を生成    
            Z2[index[j], ] = np.random.multinomial(1, psi01, 1)
            z2[index[j]] = np.argmax(Z2[index[j], ])

            # 単語を生成
            if function_flag[index[j]]==0:
                word_id[index[j]] = np.argmax(np.random.multinomial(1, phi01[z2[index[j]], ], 1).reshape(-1))
            else:
                function_word[i] = np.argmax(np.random.multinomial(1, phi02[z2[index[j]], ], 1).reshape(-1))
                word_id[index[j]] = function_word[i] + v1

        # 2単語目以降の応答変数を生成
        if j > 0:
            # 下位階層の潜在変数を生成
            Z2[index[j], ] = np.random.multinomial(1, psi02[z2[index[j-1]], ], 1)
            z2[index[j]] = np.argmax(Z2[index[j], ])

            # 単語を生成
            if function_flag[index[j]]==0:
                word_id[index[j]] = np.argmax(np.random.multinomial(1, phi01[z2[index[j]], ], 1).reshape(-1))
            else:
                function_word[i] = np.argmax(np.random.multinomial(1, phi02[z2[index[j]], ], 1).reshape(-1))
                word_id[index[j]] = function_word[i] + v1

In [ ]:
# 一部データに教師をつける
q = 3
supervised_prob = 0.5
target_syntax = np.random.choice(np.arange(syntax1), q, replace=False)
index = np.where(np.in1d(z1, target_syntax))[0].astype("int")
y = np.repeat(0, M)
y[index] = np.random.binomial(1, supervised_prob, index.shape[0])
index_y0 = np.where(y==0)[0].astype("int")
index_y1 = np.where(y==1)[0].astype("int")

# Hierarchical unsupervised phrase estimationを推定

In [ ]:
# アルゴリズムの設定
R = 1000
keep = 2
burnin = 500
skeep = int(burnin/keep)
iters = 0
disp = 50
serial_no = np.arange(N)
syntax_vec1 = np.repeat(1, syntax1)
syntax_vec2 = [np.repeat(1, syntax2[i]) for i in range(syntax1)]
phrase_matrix = scipy.sparse.csr_matrix((np.repeat(1, N), (phrase_id2, np.arange(N))), shape=(M, N))

In [ ]:
# インデックスの定義
# 単語のインデックスを定義
word_list1 = [i for i in range(v1)]
word_list2 = [i for i in range(v2)]
word_vec1 = [i for i in range(v1)]
word_vec2 = [i for i in range(v2)]
for i in range(v1):
    word_list1[i] = np.where(word_id==i)[0].astype("int")
    word_vec1[i] = np.repeat(1, word_list1[i].shape[0])
for i in range(v2):
    word_list2[i] = np.where(word_id==v1+i)[0].astype("int")
    word_vec2[i] = np.repeat(1, word_list2[i].shape[0])
    
# 先頭と末尾のインデックスを定義
index_p11 = np.where(pt_id1==0)[0].astype("int")
index_q11 = np.where(pt_id2==0)[0].astype("int")
index_p12 = np.repeat(0, L)
index_q12 = np.repeat(0, M)
for i in range(L):
    index_p12[i] = np.max(sentence_list1[i])
for i in range(M):    
    index_q12[i] = np.max(phrase_list[i])
    
# 中間のインデックスを定義
index_list_p21 = [j for j in range(max_pt1-1)]
index_list_p22 = [j for j in range(max_pt1-1)]
index_list_q21 = [j for j in range(max_pt2-1)]
index_list_q22 = [j for j in range(max_pt2-1)]
for j in range(1, max_pt1):
    index_list_p21[j-1] = np.where(pt_id1==j)[0].astype("int") - 1
    index_list_p22[j-1] = np.where(pt_id1==j)[0].astype("int")
for j in range(1, max_pt2):
    index_list_q21[j-1] = np.where(pt_id2==j)[0].astype("int") - 1
    index_list_q22[j-1] = np.where(pt_id2==j)[0].astype("int")
index_p21 = np.sort(np.hstack((index_list_p21)))
index_p22 = np.sort(np.hstack((index_list_p22)))
index_q21 = np.sort(np.hstack((index_list_q21)))
index_q22 = np.sort(np.hstack((index_list_q22)))

# 機能語のインデックスを定義
new_function = function_word.copy()
new_function[index_p12] = np.max(v2)
function_list1 = [j for j in range(v2)]
function_list2 = [j for j in range(v2)]
for j in range(v2):
    function_list1[j] = np.where(new_function==j)[0].astype("int") 
    function_list2[j] = function_list1[j] + 1

In [ ]:
# 事前分布の定義
# HMMの事前分布を定義
alpha1 = np.repeat(0.5, syntax1)
alpha21 = np.array([0.5, 0.5])
alpha22 = np.array([1.0, 1.0])
beta1 = 0.2
beta2 = 0.2

# 単語分布の事前分布
gamma1 = 0.1
gamma2 = 0.1

In [ ]:
# パラメータの真値を定義
# 推移確率のパラメータの真値
theta1 = thetat1.copy()
theta2 = thetat2.copy()
pi1 = pit1.copy()
pi2 = pit2.copy()
psi1 = psit1.copy()
psi2 = psit2.copy()

# 単語分布のパラメータの真値
phi1 = phit1.copy()
phi2 = phit2.copy()
phi = [np.hstack((phi1[j], phi2[j])) for j in range(syntax1)]

# 潜在変数の真値
Zi1 = Z1.copy()
Zi2 = Z2.copy()
z1 = np.dot(Zi1, np.arange(syntax1))
z2 = np.dot(Zi2, np.arange(np.max(syntax2)))

In [ ]:
# パラメータの初期値
# 推移確率のパラメータの初期値
theta1 = np.random.gamma(0.5, 0.5, syntax1*syntax1).reshape(syntax1, syntax1)
theta2 = np.random.gamma(0.5, 0.5, v2*syntax1).reshape(v2, syntax1)
pi1 = np.random.dirichlet(np.repeat(1.0, syntax1), 1).reshape(-1)
pi2 = np.zeros((syntax1, syntax1, v2))
for j in range(v2):
    logit = theta1 * theta2[j, ]
    pi2[:, :, j] = logit / np.sum(logit, axis=1)[:, np.newaxis]
psi1 = [np.random.dirichlet(np.repeat(5.0, syntax2[j]) , 1).reshape(-1) for j in range(syntax1)]
psi2 = [np.random.dirichlet(np.repeat(5.0, syntax2[j]), syntax2[j]) for j in range(syntax1)]

# 単語分布のパラメータの初期値
phi1 = []; phi2 = []; phi = []
for j in range(syntax1):
    phi1.append(np.random.dirichlet(np.repeat(5.0, v1), syntax2[j]))
    phi2.append(np.random.dirichlet(np.repeat(5.0, v2), syntax2[j]))
    phi.append(np.hstack((phi1[j], phi2[j])))

In [ ]:
# パラメータの格納用配列
# バーンインのインデックスを定義
RS = np.arange(skeep, int(R/keep))
rs = RS.shape[0]

# 推移確率とトピック分布の格納用配列
THETA1 = np.zeros((syntax1, syntax1, rs))
THETA2 = np.zeros((v2, syntax1, rs))
PSI1 = []
PSI2 = []

# モデルパラメータの格納用配列
PHI1 = []
PHI2 = []
PHI = []

# 潜在変数の格納用配列
SEG1 = np.zeros((rs, M), dtype="int8")
SEG2 = np.zeros((rs, N), dtype="int8")

In [ ]:
# 対数尤度の基準値
# ユニグラムモデルの対数尤度
freq = np.unique(word_id, return_counts=True)
par1 = np.repeat(0.0, v1)
par2 = np.repeat(0.0, v2)
par1[freq[0][freq[0] < v1]] = freq[1][freq[0] < v1]
par2[freq[0][freq[0] >= v1] - v1] = freq[1][freq[0] >= v1]
LLst1 = np.sum(np.log((par1 / np.sum(par1))[word_id[word_id < v1]]))
LLst2 = np.sum(np.log((par2 / np.sum(par2))[word_id[word_id >= v1] - v1]))
LLst = LLst1 + LLst2
print(np.round([LLst1, LLst2, LLst], 1))

# 真値での対数尤度
LLbest = 0.0; LLbest1 = 0.0; LLbest2 = 0.0
syntax_long = Z1[phrase_id2, ]
for j in range(syntax1):
    index = np.where(syntax_long[:, j])[0].astype("int")
    LLho = np.log(np.dot(Z2[index, ] * (phit[j].T)[word_id[index], ], syntax_vec2[j]))
    LLbest1 += np.sum(LLho[word_id[index] < v1])
    LLbest2 += np.sum(LLho[word_id[index] >= v1])
    LLbest += np.sum(LLho)
print(np.round([LLbest1, LLbest2, LLbest], 1))

## パラメータを推定

In [ ]:
# ギブスサンプリングでパラメータをサンプリング
start_time = time.time()
for rp in range(R):
    
    # 上位階層のsyntaxごとに下位階層のsyntaxを生成
    # 上位階層のパラメータの格納用配列
    Syntax = np.zeros((N, np.max(syntax2), syntax1), dtype="int")
    Lho = np.zeros((N, syntax1))

    # syntaxに応じた下位階層のsyntaxを生成
    for i in range(syntax1):

        # 潜在変数の格納用配列
        Posterior2 = np.zeros((N, syntax2[i]))
        Zi2 = np.zeros((N, syntax2[i]), dtype="int")
        z2 = np.repeat(0, N)

        # 下位階層の尤度を定義
        phi_long = (phi[i].T)[word_id, ]

        # 語順に応じてsyntaxを生成
        for j in range(max_pt2):

            # フレーズの先頭のsyntaxを生成
            index = pt_list21[j]
            if j==0:
                # 事後分布から潜在確率を定義
                prior = psi1[i]
                Posterior2 = prior * phi_long[index, ]
                Prob2 = Posterior2 / np.dot(Posterior2, syntax_vec2[i])[:, np.newaxis]

                # 多項分布からsyntaxを生成
                res = rmnom(Prob2, pt_n2[j], syntax2[i], 1)
                z2[index] = res[0]
                Zi2[index, :syntax2[i]] = res[1]

                # 上位階層の尤度を定義
                Lho[index, i] = np.dot(res[1] * Posterior2, syntax_vec2[i])

            # フレーズの2単語目以降のsyntaxを生成
            else:
                # 事後分布から潜在確率を定義
                prior = psi2[i][z2[pt_list20[j]], ]
                Posterior2 = prior * phi_long[index]
                Prob2 = Posterior2 / np.dot(Posterior2, syntax_vec2[i])[:, np.newaxis]

                # 多項分布からsyntaxを生成
                res = rmnom(Prob2, pt_n2[j], syntax2[i], 1)
                z2[index] = res[0]
                Zi2[index, :syntax2[i]] = res[1]

                # 上位階層の尤度を定義
                Lho[index, i] = np.dot(res[1] * Posterior2, syntax_vec2[i])

        # 潜在変数を格納
        Syntax[:, :, i] = Zi2


    # 事後分布から上位階層のsyntaxを生成
    # 推移確率の事前分布を定義
    pi_dt1 = np.full((M, syntax1), 1/syntax1); pi_dt2 = pi_dt1.copy()
    pi_dt1[index_p11, ] = np.full((L, syntax1), pi1)   # フレーズの先頭の混合率
    pi_dt1[index_p22, ] = pi2[z1[index_p21], :, function_word[index_p21]]   # 1フレーズ前の混合率
    pi_dt2[index_p21, ] = (pi2.T)[function_word[index_p22], z1[index_p22], ]   # 1フレーズ後の混合率

    # syntaxごとの事後分布を定義
    LL = np.dot(phrase_matrix, scipy.sparse.csr_matrix(np.log(Lho))).toarray()
    Posterior1 = pi_dt1 * pi_dt2 * np.exp(LL - np.max(LL, axis=1)[:, np.newaxis])

    # 多項分布からsyntaxを生成
    Prob1 = Posterior1 / np.dot(Posterior1, syntax_vec1)[:, np.newaxis]
    res = rmnom(Prob1, M, syntax1, 1)
    Zi1 = res[1]
    z1 = res[0]

    # 上位階層のsyntaxに対応する下位階層のsyntaxを取得
    syntax_long1 = Zi1[phrase_id2, ]
    Zi2 = Syntax[serial_no, :, z1[phrase_id2]]
    z2 = np.dot(Zi2, np.arange(np.max(syntax2)))


    ## 事前分布のパラメータを更新
    # 推移確率の初期パラメータを更新
    rf1 = np.sum(Zi1[index_p11, ], axis=0) + alpha1
    rf2 = np.dot(syntax_long1[index_q11, ].T, Zi2[index_q11, ]) + beta1
    pi1 = np.random.dirichlet(rf1, 1).reshape(-1)
    for j in range(syntax1):
        psi1[j] = np.random.dirichlet(rf2[j, ], 1).reshape(-1)

    # 上位階層の推移確率のパラメータを更新
    # グローバルパラメータの格納用配列
    r1 = np.zeros((syntax1, syntax1))
    s1 = np.zeros((syntax1, syntax1))

    # 機能語ごとにパラメータを更新
    for i in range(v2):

        # データを定義
        index1 = function_list1[i]
        index2 = function_list2[i]
        y = np.dot(Zi1[index1, ].T, Zi1[index2, ])
        y_row = np.sum(y, axis=1)[:, np.newaxis]

        # ローカルパラメータを更新
        r2  = np.sum(y, axis=0) + beta2
        s2 = np.sum(y_row * theta1, axis=0) + beta2
        theta2[i, ] = np.random.gamma(r2, 1/s2, syntax1)

        # ローカル単位の頻度の和を更新
        r1 += y
        s1 += y_row * theta2[i, ]

    # グローバルパラメータを更新
    r1 = r1 + beta2
    s1 = s1 + beta2
    theta1 = np.random.gamma(r1, 1/s1)

    # パラメータを推移確率に変換
    pi2 = np.zeros((syntax1, syntax1, v2))
    for j in range(v2):
        logit = theta1 * theta2[j, ]
        pi2[:, :, j] = logit / np.sum(logit, axis=1)[:, np.newaxis]

    # 下位階層の推移確率のパラメータを更新
    # データの定義
    target_z1 = Zi2[index_q21, ]
    target_z2 = Zi2[index_q22, ]
    target_syntax = syntax_long1[index_q22, ]

    # 上位階層に応じて下位階層の推移確率を更新
    for j in range(syntax1):
        rf = np.dot((target_syntax[:, j][:, np.newaxis] * target_z1).T, target_syntax[:, j][:, np.newaxis] * target_z2) + beta2
        psi2[j] = np.array([np.random.dirichlet(rf[g, ], 1).reshape(-1) for g in range(syntax2[j])])


    # 単語分布のパラメータを更新
    # ディリクリ分布のパラメータの格納用配列
    wsum1 = [np.zeros((syntax2[j], v1)) for j in range(syntax1)]
    wsum2 = [np.zeros((syntax2[j], v2)) for j in range(syntax1)]

    # 一般語の単語分布を更新
    for i in range(v1):
        # データを定義
        index = word_list1[i]
        target_z = Zi2[index, ].T
        target_syntax = syntax_long1[index, ]

        # ディリクレ分布のパラメータを定義
        for j in range(syntax1):
            wsum1[j][:, i] = np.dot(target_syntax[:, j] * target_z, word_vec1[i]) + gamma1

    # 機能語の単語分布を更新
    for i in range(v2):
        # データを定義
        index = word_list2[i]
        target_z = Zi2[index, ].T
        target_syntax = syntax_long1[index, ]

        # ディリクレ分布から新しいパラメータを更新
        for j in range(syntax1):
            wsum2[j][:, i] = np.dot(target_syntax[:, j] * target_z, word_vec2[i]) + gamma2

    # ディリクリ分布から新しいパラメータを更新
    for i in range(syntax1):
        phi1[i] = np.array([np.random.dirichlet(wsum1[i][j, ], 1).reshape(-1) for j in range(syntax2[i])])
        phi2[i] = np.array([np.random.dirichlet(wsum2[i][j, ], 1).reshape(-1) for j in range(syntax2[i])])
        phi[i] = np.hstack((phi1[i], phi2[i]))


    # サンプリング結果の保存と表示
    # パラメータの格納
    if (rp%keep==0) & (rp >= burnin):
        mkeep = int(rp/keep) - skeep

        # モデルパラメータの格納
        THETA1[mkeep, ] = theta1
        THETA2[:, :, mkeep] = theta2
        PSI1.append(psi1)
        PSI2.append(psi2)
        PHI1.append(phi1)
        PHI2.append(phi2)
        PHI.append(phi)

        # 潜在変数の格納
        SEG1[mkeep, ] = z1
        SEG2[mkeep, ] = z2 

    # 対数尤度の更新と結果の表示
    if rp%disp==0:
        # 経過時間を取得
        intermediate_time = time.time()
        elapsed_time = (intermediate_time - start_time) / 60

        # 対数尤度の更新
        LL = 0.0; LL1 = 0.0; LL2 = 0.0
        syntax_long = Zi1[phrase_id2, ]
        for j in range(syntax1):
            index = np.where(syntax_long[:, j])[0].astype("int")
            LLho = np.log(np.dot(Zi2[index, ] * (phi[j].T)[word_id[index], ], syntax_vec2[j]))
            LL1 += np.sum(LLho[word_id[index] < v1])
            LL2 += np.sum(LLho[word_id[index] >= v1])
            LL += np.sum(LLho)

        #サンプリング結果を確認
        print(rp)
        print("経過時間: {}".format(elapsed_time))
        print(np.round([LL, LLst, LLbest], 1))